In [1]:
import os

In [2]:
%pwd

'g:\\Il mio Drive\\PROJECTS\\Kidney-Tumor-Detection-End-To-End\\research'

Currently in Research folder -> I want to create the artifacts folder (config.YAML) in the general repo and not inside Research folder.

In [3]:
os.chdir("../")

In [4]:
%pwd

'g:\\Il mio Drive\\PROJECTS\\Kidney-Tumor-Detection-End-To-End'

Perfect.

---

### Entity -> Return type of any function:

In [5]:
from dataclasses import dataclass
from pathlib import Path

# dataclass to consider this not as a Python class but as an Entity
#a sort of "Class variable" from which you can access these components from other files
# in this case config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
# these are all the things mentioned in the config.yaml and I defined the outputs of these components

Link the params and config YAML files here src\CNN_Classifier\constants\__init__.py

In [6]:
from CNN_Classifier.constants import *          # import the YAML linking
from CNN_Classifier.utils.common import read_yaml, create_directories

Now:

5. Update the configuration manager in src config

This method will be the one I will use to manage my entire variable environment.

In [7]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        # read yaml files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        # using ConfigBox syntax to access the value "artifacts" from the ke "artifacts_root"


    # DataIngestionConfig return type created before as a dataclass/entity
    # in fact this function will only return these values root_dir, source_URL, local_data_file, unzip_dir
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        # extract data using ConfigBox syntax
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

6. Update the components

In [11]:
import os
import zipfile      # to unzip my data
import gdown
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [12]:
# Data Ingestion class that will take as input my DataIngestionConfig (output of previous part)
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config        # has the 4 values

    # download the data using gdown and the config values
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    # unzip data in unzip_dir: artifacts/data_ingestion
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-16 20:19:08,324: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-16 20:19:08,333: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-16 20:19:08,335: INFO: common: created directory at: artifacts]
[2023-11-16 20:19:08,338: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-16 20:19:08,340: INFO: 690492142: Downloading data from https://drive.google.com/file/d/1Sm3W06lYtO1f3i6RM0z3iiET1zdBj6AO/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1Sm3W06lYtO1f3i6RM0z3iiET1zdBj6AO
From (redirected): https://drive.google.com/uc?/export=download&id=1Sm3W06lYtO1f3i6RM0z3iiET1zdBj6AO&confirm=t&uuid=caabc904-caae-4c4a-9f4e-216782a67879
To: g:\Il mio Drive\PROJECTS\Kidney-Tumor-Detection-End-To-End\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [00:17<00:00, 54.7MB/s] 

[2023-11-16 20:19:27,490: INFO: 690492142: Downloaded data from https://drive.google.com/file/d/1Sm3W06lYtO1f3i6RM0z3iiET1zdBj6AO/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloaded data.

Now copy the pieces of this notebook into each subfolder accordin to workflow in the readme.